# Worksheet 05

Name:  
UID: 

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

cluster 1: [0, 0.5]
cluster with 2: [1.5, 2, 6, 6.5, 7]

mean of cluster 1: 0.25
mean of cluster 2: 4.6

cluster centroids: [0.25, 4.6]
cluster 1: [0, 0.5, 1.5, 2]
cluster 2: [6, 6.5, 7]

mean of cluster 1: 1.0
mean of cluster 2: 6.5

cluster 1: [0, 0.5, 1.5, 2]
cluster 2: [6, 6.5, 7]

converged

b) Describe in plain english what the cost function for k means is.

cost function is the distance from a certain point to any of the centroids

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Because the clusters depends on which centroids are chosen (often chosen randomly)

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, because the algorithm will always converge to a local minimum

e) Follow along in class the implementation of Kmeans. Use proof by contradiction to show that the cost function never increases:

The cost function is the sum of the distances from each point to its centroid. If the cost function increases, then the distance from a point to its centroid increases. But this is impossible because the centroid is the point that minimizes the distance to all other points.

In [5]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize (self):
        return self.data[np.random.choice(len(self.data), self.k, replace=False)]

    def assign(self, centers):
        new_assignment = [-1 for _ in range(len(self.data))]
        for i, x in enumerate(self.data):
            new_assignment[i] = np.argmin([np.linalg.norm(x - c) for c in centers])
        if new_assignment == self.assignment:
            return True
        else:
            self.assignment = new_assignment
            return False

    def update(self, centers):
        new_centers = np.zeros((self.k, 2))
        for i in range(self.k):
            new_centers[i] = np.mean(self.data[np.where(np.array(self.assignment) == i)], axis=0)
        return new_centers

    def lloyds(self):
        centers = self.data[np.random.choice(len(self.data), self.k, replace=False)]
        self.snap(centers)
        self.assign(centers)
        self.snap(centers)
        while True:
            centers = self.update(centers)
            self.snap(centers)
            if self.assign(centers):
                break

        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)